In [1]:
import numpy as np
import cv2 as cv
import os

import matplotlib.pyplot as plt

from IPython.display import Video
from ipywebrtc import VideoStream

In [2]:
VideoStream.from_file("/mnt/data/small_video_clips/raws_Breaking_Bad_s01e01/raws_Breaking_Bad_s01e01-9.mkv")

VideoStream(video=Video(value=b'\x1aE\xdf\xa3\x01\x00\x00\x00\x00\x00\x00#B\x86\x81\x01B\xf7\x81\x01B\xf2\x81\…

In [3]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="/mnt/data/small_video_clips/raws_Breaking_Bad_s01e01/raws_Breaking_Bad_s01e01-9.mkv" type="video/mp4">
    </video>
""")

In [4]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines+0.5)
    vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    cv.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [5]:
def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bgr

In [6]:
def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv.remap(img, flow, None, cv.INTER_LINEAR)
    return res

In [7]:
def display(frame):    
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

In [8]:
""" input and output file paths """
path = '/mnt/data/small_video_clips/files_to_test_motion_algos/'

out_path_FB = '/mnt/data/small_video_clips/FB_dense_OF_out/'
out_path_DenseRLOF = '/mnt/data/small_video_clips/dense_RLOF_OF_out/'
out_path_OF_SF = '/mnt/data/small_video_clips/OF_SF_out/'
out_path_STD = '/mnt/data/small_video_clips/OF_SparseToDense/'
out_path_DeepFlow = '/mnt/data/small_video_clips/DeepFlow/'
out_path_tv1Flow = '/mnt/data/small_video_clips/tv1flow/'
out_path_vOFr = '/mnt/data/small_video_clips/vOFr/'
out_path_PCAFlow = '/mnt/data/small_video_clips/PCAFlow'
out_path_SparseRLOF = '/mnt/data/small_video_clips/SparseFlow/'

In [9]:
""" creates the output directories if they are not created yet """
def create_output_directories():
    if not os.path.exists(out_path_FB):
        os.makedirs(out_path_FB)
    if not os.path.exists(out_path_DenseRLOF):
        os.makedirs(out_path_DenseRLOF)
    if not os.path.exists(out_path_OF_SF):
        os.makedirs(out_path_OF_SF)
    if not os.path.exists(out_path_STD):
        os.makedirs(out_path_STD)
    if not os.path.exists(out_path_DeepFlow):
        os.makedirs(out_path_DeepFlow)
    if not os.path.exists(out_path_tv1Flow):
        os.makedirs(out_path_tv1Flow)
    if not os.path.exists(out_path_vOFr):
        os.makedirs(out_path_vOFr)
    if not os.path.exists(out_path_PCAFlow):
        os.makedirs(out_path_PCAFlow)
    if not os.path.exists(out_path_SparseRLOF):
        os.makedirs(out_path_SparseRLOF)    

In [10]:
""" Enable the approach here """
isFarneBack = False
isDeepFlow = False
isDenseRLOF = False
isTV1Flow = True
isPCAFlow = False
isOFSimpleFlow = False
isSparseRLOF = False
isOFSparseToDense = False
isvOFr = False

target_size = (224, 224 #target size

In [11]:
""" create the of object here based on approach choosen """
def create_OF_object():
    if isDeepFlow:
        optFlow = cv.optflow.createOptFlow_DeepFlow()
    elif isDenseRLOF:
        optFlow = cv.optflow.createOptFlow_DenseRLOF()
    elif isTV1Flow:
        optFlow = cv.optflow.createOptFlow_DualTVL1()
    elif isFarneBack:
        """ parameters """
        numLevels = 3 #5
        pyrScale = 0.5
        fastPyramids = False
        winSize = 9 #13
        numIters = 10
        polyN = 5
        polySigma = 1.2
        flags = 0
        optFlow = cv.FarnebackOpticalFlow_create(numLevels, pyrScale, fastPyramids, winSize, numIters, polyN, polySigma, flags)
        #optFlow = cv.optflow.createOptFlow_Farneback(numLevels, pyrScale, fastPyramids, winSize, numIters, polyN, polySigma, flags)
    elif isPCAFlow:
        optFlow = cv.optflow.createOptFlow_PCAFlow()
    elif isSimpleFlow:
        optFlow = cv.optflow.createOptFlow_SimpleFlow()
    elif isSparseRLOF:
        optFlow = cv.optflow.createOptFlow_SparseRLOF()
    elif isOFSparseToDense:
        optFlow = cv.optflow.createOptFlow_SparseToDense()
    elif isvOFr:
        optFlow = cv.VariationalRefinement_create()

    return optFlow

In [12]:
""" create output file name here to write the OF output into the file """
def create_output_filename(dst_file_name):
    if isFarneBack:
        out_file_path = os.path.join(out_path_FB, dst_file_name)
    elif isDenseRLOF:
        out_file_path = os.path.join(out_path_DenseRLOF, dst_file_name)
    elif isOFSimpleFlow:
        out_file_path = os.path.join(out_path_OF_SF, dst_file_name)
    elif isOFSparseToDense:
        out_file_path = os.path.join(out_path_STD, dst_file_name)
    elif isDeepFlow:
        out_file_path = os.path.join(out_path_DeepFlow, dst_file_name)
    elif isTV1Flow:
        out_file_path = os.path.join(out_path_tv1Flow, dst_file_name)
    elif isvOFr:
        out_file_path = os.path.join(out_path_vOFr, dst_file_name)
    elif isPCAFlow:
        out_file_path = os.path.join(out_path_PCAFlow, dst_file_name)
    elif isSparseRLOF:
        out_file_path = os.path.join(out_path_SparseRLOF, dst_file_name)
        
    return out_file_path

In [13]:
def get_tv1_parameters(optflow):
    epsilon = optflow.getEpsilon()
    gamma = optflow.getGamma()
    innIter = optflow.getInnerIterations()
    lambda_val = optflow.getLambda()
    medFilter = optflow.getMedianFiltering()
    outIter = optflow.getOuterIterations()
    scaleNum = optflow.getScalesNumber()
    scalestep = optflow.getScaleStep()
    tau = optflow.getTau()
    theta = optflow.getTheta()
    initFlow = optflow.getUseInitialFlow()
    warpNum = optflow.getWarpingsNumber()
    
    #print("params:", epsilon, gamma, innIter, lambda_val, medFilter, outIter, scaleNum, scalestep, tau, theta, initFlow, warpNum)

In [14]:
def set_tv1_parameters(optflow):
    epsilon = 0.0 
    optflow.setEpsilon(epsilon)
    gamma = 0.0
    optflow.setGamma(gamma)
    innIter = 0
    optflow.setInnerIterations(innIter)
    lambda_val = 0.0
    optflow.setLambda(lambda_val)
    medFilter = 0
    optflow.setMedianFiltering(medFilter)
    outIter = 0
    optflow.setOuterIterations(outIter)
    scaleNum = 0
    optflow.setScalesNumber(scaleNum)
    scalestep = 0.0
    optflow.setScaleStep(scalestep)
    tau = 0.0
    optflow.setTau(tau)
    theta = 0.0
    optflow.setTheta(theta)
    initFlow = 0
    optflow.setUseInitialFlow(initFlow)
    warpNum = 0
    optflow.setWarpingsNumber(warpNum)

In [15]:
def calc_optical_flow(optFlow, res_prev_frame, res_curr_frame, prev_gray, curr_gray):
    if isFarneBack:
        #flow = cv.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        flow = optFlow.calc(prev_gray, curr_gray, flow_field)

    elif isDenseRLOF:
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        flow = cv.optflow.calcOpticalFlowDenseRLOF(res_prev_frame, res_curr_frame, flow_field)

    elif isOFSimpleFlow:
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        flow = cv.optflow.calcOpticalFlowSF(res_prev_frame, res_curr_frame, 3, 2, 4, 4.1, 25.5, 18, 55.0, 25.5, 0.35, 18, 55.0, 25.5, 10)

    elif isOFSparseToDense:
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        flow = cv.optflow.calcOpticalFlowSparseToDense(res_prev_frame, res_curr_frame, flow_field);

    elif isDeepFlow:
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        flow = optFlow.calc(prev_gray, curr_gray, flow_field)

    elif isTV1Flow:
        flow_field = np.zeros((target_size[0], target_size[1], 2))
        get_tv1_parameters(optFlow)
        #set_tv1_parameters(optFlow)
        flow = optFlow.calc(prev_gray, curr_gray, flow_field)

    elif isvOFr:
        flow_u = np.zeros(target_size[0], target_size[1], dtype=np.float32)
        flow_v = np.zeros(target_size[0], target_size[1], dtype=np.float32)

        flow_u, flow_v = optFlow.calcUV(prev_gray, curr_gray, flow_u, flow_v)
        flow = np.dstack([flow_u, flow_v])
        
    elif isSparseRLOF:
        print("in isSparseRLOF, Yet to add support")
        
    elif isPCAFlow:
        print("in isPCAFlow, yet to add support")
        
    return flow

In [16]:
def main():
    create_output_directories()
        
    optFlow = create_OF_object()
    
    """ iter for all video clips """
    for fname in os.listdir(path):
        src_file_path = os.path.join(path, fname)

        """ load video file """
        cam = cv.VideoCapture(src_file_path)
    
        """ read first frame and convert to gray """
        _ret, prev_frame = cam.read()
        frame_height, frame_width = prev_frame.shape[:2]
        
        """ resize it to new width and height of I3D model"""
        res_prev_frame = cv.resize(prev_frame, target_size)
        
        """ convert from BGR to RGB """
        prev_gray = cv.cvtColor(res_prev_frame, cv.COLOR_BGR2GRAY)

        """create videowriter handle to create a output video file """
        base, ext = fname.split('.')
        dst_file_name = base+'.avi'
        
        out_file_path = create_output_filename(dst_file_name)
        writer = cv.VideoWriter(out_file_path, cv.VideoWriter_fourcc('M','J','P','G'), 10, target_size)

        """ choose the flow display method """
        show_hsv = True
        show_glitch = False
        cur_glitch = prev_frame.copy()

        """ iterate for all the frames and compute the flow from every two frames using selected approach """
        while True:
            _ret, curr_frame = cam.read()
            if _ret == False:
                break
            res_curr_frame = cv.resize(curr_frame, target_size)
            curr_gray = cv.cvtColor(res_curr_frame, cv.COLOR_BGR2GRAY)
            
            flow = calc_optical_flow(optFlow, res_prev_frame, res_curr_frame, prev_gray, curr_gray)
            if show_hsv:
                bgr = draw_hsv(flow)
                #display(bgr)
                writer.write(bgr)

            if show_glitch:
                cur_glitch = warp_flow(cur_glitch, flow)
                display(cur_glitch)
                
            prev_gray = curr_gray
            res_prev_frame = res_curr_frame
                
        cam.release()
        writer.release()

In [17]:
if __name__== "__main__":
    print("__doc__")
    main()
    cv.destroyAllWindows()

__doc__
